In [1]:
import pandas as pd
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
import calendar
import re
import numpy as np
%matplotlib inline 

In [2]:
from p_values import Variance_Inflation_Factor
def variance_inflation_factor(x):
    columns = x.columns
    VIF = Variance_Inflation_Factor(x)
    df_vif = pd.DataFrame({"vif": VIF})
    df_vif = df_vif.set_index(columns)
    return df_vif

In [3]:
prod3channel = [22152,
2158,
300,
2451,
2449,
22278,
9199,
3690,
22150,
9189]#add 9189

In [4]:
max_model = pd.read_excel("maximum_likes_model_posting_lastweek_v1.xlsx").set_index(["yymmdd"])

dt_product = pd.read_excel("3channel_wsm_ps_202001123-tv.xlsx").set_index(["pd_code","yymmdd"])
store_display = pd.read_excel("store_display.xlsx")

In [5]:
dt_product["product_only"].max()

37

In [6]:
dt_product.reset_index()["pd_code"].value_counts()

2449      634
2451      634
5152      634
2158      634
309       634
222789    634
Name: pd_code, dtype: int64

In [7]:
dt_product["avg_likes_product_only"].max()

692.5714285714283

In [8]:
dt_product.columns

Index(['product_model', 'product_only', 'likes_by_product_model',
       'likes_by_product_only', 'spillover_product_model',
       'spillover_product_only', 'total_likes_by_product_model',
       'total_likes_by_product_only', 'total_spillover', 'numProduct_mention',
       'likes_by_spillover_product_model', 'likes_by_spillover_product_only',
       'total_likes_by_spillover', 'avg_likes_product_model',
       'avg_likes_product_only', 'avg_likes_spillover_product_model',
       'avg_likes_spillover_product_only', 'avg_likes_total_spillover'],
      dtype='object')

In [9]:
sales_data = pd.read_excel("daily_analysis_oyof_sales_data_v3.xlsx").fillna(0).set_index("pd_code")
#daily_analysis_pc_sales_data_final20200907v1.xlsx
# daily_analysis_mobile_sales_data_final20200907v1
#daily_analysis_oyof_sales_data_v3.xlsx for oyof use 
# use daily_analysis_mobilepc_sales_data_20200907v3.xlsx for mobilepc
#if offline, drop week
#mobile= pd.read_excel("daily_analysis_sales_data_v1.xlsx").set_index("pd_code")



In [111]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6400 entries, 22152 to 22150
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   yymmdd        6400 non-null   datetime64[ns]
 1   sales_qt      6400 non-null   float64       
 2   buy_offer     6400 non-null   float64       
 3   buy_original  6400 non-null   float64       
 4   Week          6400 non-null   int64         
 5   Year          6400 non-null   int64         
 6   Days          6400 non-null   object        
 7   price_dc      6400 non-null   float64       
 8   price_reg     6400 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(2), object(1)
memory usage: 629.0+ KB


In [10]:
#pc.append(mobile).set_index("pd_code").sort_values(by=("Week")).loc[300]

In [11]:
#total_sales= pc.append(mobile).groupby(["pd_code","Week"]).sum()

In [12]:
#make sure your data contains all the 9 products
pc = sales_data.loc[prod3channel]

In [13]:
total_sales = pc.reset_index()

In [14]:
total_sales["pd_code"].value_counts()

300      640
2449     640
22150    640
22278    640
9189     640
2451     640
22152    640
2158     640
9199     640
3690     640
Name: pd_code, dtype: int64

In [15]:
total_sales.loc[total_sales["price_dc"]==total_sales["price_dc"].max()]

,pd_code,yymmdd,sales_qt,buy_offer,buy_original,Week,Year,Days,price_dc,price_reg
1918,300,2016-07-27,48.0,1.349077e+06,1776000.0,30,2016,Wednesday,28105.769231,37000.0


In [16]:
total_sales.loc[total_sales["sales_qt"]>=600]

,pd_code,yymmdd,sales_qt,buy_offer,buy_original,Week,Year,Days,price_dc,price_reg
0,22152,2016-03-04,934.0,2.836100e+06,3736000.0,9,2016,Friday,3036.509636,4000.0
1,22152,2016-09-01,1699.0,5.165000e+06,6796000.0,35,2016,Thursday,3040.023543,4000.0
2,22152,2016-03-02,866.0,2.634800e+06,3464000.0,9,2016,Wednesday,3042.494226,4000.0
3,22152,2016-03-03,954.0,2.905200e+06,3816000.0,9,2016,Thursday,3045.283019,4000.0
4,22152,2016-12-02,1111.0,3.383600e+06,4444000.0,48,2016,Friday,3045.544554,4000.0
...,...,...,...,...,...,...,...,...,...,...
2894,2449,2017-11-30,1952.0,2.891468e+07,62464000.0,100,2017,Thursday,14812.849427,32000.0
3838,22278,2017-11-30,1952.0,1.542080e+07,15420800.0,100,2017,Thursday,7900.000000,7900.0
3862,9199,2017-11-30,4064.0,3.470031e+07,60960000.0,100,2017,Thursday,8538.461538,15000.0
4804,3690,2017-11-30,2176.0,2.154240e+07,26112000.0,100,2017,Thursday,9900.000000,12000.0


In [17]:
#make sure your data contains all the 9 products
total_sales["pd_code"].unique().sort() == np.array(prod3channel).sort()

True

In [18]:
total_sales.columns

Index(['pd_code', 'yymmdd', 'sales_qt', 'buy_offer', 'buy_original', 'Week',
       'Year', 'Days', 'price_dc', 'price_reg'],
      dtype='object')

In [19]:
total_sales.shape

(6400, 10)

In [20]:
#total_sales["pd_code"].unique()

In [21]:
def new_pdcode(cd):
    if cd in [300,9199,9189]:
        return 309
    elif cd in [22150,22151,22152]:
        return 5152
    elif cd in [22164, 22165]:
        return 6465
    elif cd in [22278]:
        return 222789
    else:
        return cd


In [22]:
total_sales["ori_pdcode"] = total_sales["pd_code"].astype(str)
total_sales["pd_code"]=total_sales["pd_code"].apply(new_pdcode)
store_display["pd_code"] = store_display["pd_code"].apply(new_pdcode)

In [23]:
total_sales["pd_code"].value_counts()

309       1920
5152      1280
2449       640
222789     640
2451       640
2158       640
3690       640
Name: pd_code, dtype: int64

In [24]:

store_display= store_display.set_index(["pd_code","Week"])

In [25]:
total_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6400 entries, 0 to 6399
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   pd_code       6400 non-null   int64         
 1   yymmdd        6400 non-null   datetime64[ns]
 2   sales_qt      6400 non-null   float64       
 3   buy_offer     6400 non-null   float64       
 4   buy_original  6400 non-null   float64       
 5   Week          6400 non-null   int64         
 6   Year          6400 non-null   int64         
 7   Days          6400 non-null   object        
 8   price_dc      6400 non-null   float64       
 9   price_reg     6400 non-null   float64       
 10  ori_pdcode    6400 non-null   object        
dtypes: datetime64[ns](1), float64(5), int64(3), object(2)
memory usage: 550.1+ KB


In [26]:
dt_model = pd.read_excel("wsm_model_20200928#2.xlsx").merge(max_model, on="yymmdd")

#dt_product["pd_code"] = dt_product["pd_code"].apply(lambda x:int(x))

In [27]:
dt_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 634 entries, 0 to 633
Data columns (total 35 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   yymmdd                                 634 non-null    datetime64[ns]
 1   avg_numBrand                           634 non-null    float64       
 2   avg_numModel                           634 non-null    float64       
 3   avg_numOYBrand                         634 non-null    float64       
 4   avg_numPos-Neg                         634 non-null    float64       
 5   avg_Colorfulness_Picture               634 non-null    float64       
 6   avg_Saturation_Picture                 634 non-null    float64       
 7   avg_Clarity_Picture                    634 non-null    float64       
 8   avg_Brightness_Picture                 634 non-null    float64       
 9   avg_Warm Hue_Picture                   634 non-null    float64   

In [28]:
total_sales.merge(dt_model,how="left").head()

,pd_code,yymmdd,sales_qt,buy_offer,buy_original,Week,Year,Days,price_dc,price_reg,...,likes_by_model_only,avg_likes_by_model_only,avg_len_text_total,avg_tag_numb,avg_happiness,avg_faces,avg_image_size_kilobytes,total_posting_this_day,len_text_total,max_likes_by_model_only
0,5152,2016-03-04,934.0,2836100.0,3736000.0,9,2016,Friday,3036.509636,4000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5152,2016-09-01,1699.0,5165000.0,6796000.0,35,2016,Thursday,3040.023543,4000.0,...,243.0,243.00,286.500000,14.500000,0.000000,0.250000,51.36825,4.0,1146.0,0.0
2,5152,2016-03-02,866.0,2634800.0,3464000.0,9,2016,Wednesday,3042.494226,4000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5152,2016-03-03,954.0,2905200.0,3816000.0,9,2016,Thursday,3045.283019,4000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5152,2016-12-02,1111.0,3383600.0,4444000.0,48,2016,Friday,3045.544554,4000.0,...,836.0,52.25,128.636364,8.545455,0.121212,0.848485,57.14497,33.0,4245.0,536.0


In [29]:
#blockA = merging total sales with data model
blockAB= total_sales.merge(dt_model,how="left")

In [30]:
blockAB.shape

(6400, 45)

In [31]:
blockAB["sales_qt"].max()

11488.0

# add other sales



In [32]:

othersales = pd.read_excel("daily_analysis_mobilepc_sales_data_20200907v3.xlsx").groupby(["pd_code","yymmdd"]).sum().loc[:,["sales_qt","price_dc"]].rename(columns={"sales_qt":"other_sales_qt","price_dc":"other_price_dc"}).reset_index()
# "daily_analysis_oyof_sales_data_v3.xlsx"
# use daily_analysis_mobilepc_sales_data_20200907v3 for mobilepc
#pc_sales = pd.read_excel("daily_analysis_pc_sales_data_v2.xlsx")
#mobile_sales = pd.read_excel("daily_analysis_mobile_sales_data_v2.xlsx")
#othersales = pc_sales.append(mobile_sales).groupby(["pd_code","yymmdd"]).sum().loc[:,["sales_qt","price_dc"]].rename(columns={"sales_qt":"other_sales_qt","price_dc":"other_price_dc"}).reset_index()

In [33]:
othersales["pd_code"].value_counts()

300      640
2449     640
22150    640
22278    640
2451     640
22152    640
2158     640
9199     640
3690     640
Name: pd_code, dtype: int64

In [34]:
othersales["pd_code"] = othersales["pd_code"].astype(str)

In [35]:
sales_combined = blockAB.merge(othersales, left_on=["ori_pdcode","yymmdd"], right_on=["pd_code","yymmdd"]).drop(columns=["pd_code_y"]).rename(columns={"pd_code_x":"pd_code"})

In [112]:
sales_combined = blockAB

In [113]:
sales_combined["sales_qt"].max()

11488.0

In [114]:
sales_combined["pd_code"].value_counts()

309       1920
5152      1280
2449       640
222789     640
2451       640
2158       640
3690       640
Name: pd_code, dtype: int64

In [115]:
sales_cb = sales_combined.set_index(["pd_code","yymmdd"])

In [116]:
#blockAB.to_excel("blockAB.xlsx")

In [117]:
# = pd.read_excel("3channel_wsm_ps_20200907.xlsx")
#dtp_sample300 = dtp_sample.loc[dtp_sample["pd_code"]==309].set_index(["pd_code","yymmdd"])

In [118]:
#dtp_sample300.to_excel("dtp_sample300.xlsx")

In [119]:
#sample_300 = sales_cb.sample(n=10)

In [120]:
#sample_300.head()

In [121]:
#sample_300.join(dtp_sample300, how="left").to_excel("check_merging_productposting.xlsx")

In [122]:
dt_product.loc[222789].shape

(634, 18)

In [123]:
sls_reg = sales_cb.join(dt_product,how="left")

In [124]:
sls_reg["product_only"].max()

37.0

In [125]:
sls_reg["sales_qt"].max()

11488.0

In [126]:
sls_reg.head()

sales_qt     buy_offer  buy_original  Week  Year  \
pd_code yymmdd                                                         
309     2016-03-01     138.0  2.923569e+06     5106000.0     9  2016   
        2016-03-01     136.0  1.161231e+06     2040000.0     9  2016   
        2016-03-01      22.0  4.400000e+05      440000.0     9  2016   
        2016-03-02     159.0  3.391477e+06     5883000.0     9  2016   
        2016-03-02     154.0  1.314923e+06     2310000.0     9  2016   

                         Days      price_dc  price_reg ori_pdcode  \
pd_code yymmdd                                                      
309     2016-03-01    Tuesday  21185.284281    37000.0        300   
        2016-03-01    Tuesday   8538.461538    15000.0       9199   
        2016-03-01    Tuesday  20000.000000    20000.0       9189   
        2016-03-02  Wednesday  21330.043541    37000.0        300   
        2016-03-02  Wednesday   8538.461538    15000.0       9199   

                    avg_numBrand  ...  total_spillover  numProduct_mention  \
pd_code yymmdd                    ...                                        
309     2016-03-01           NaN  ...              NaN                 NaN   
        2016-03-01           NaN  ...              NaN                 NaN   
        2016-03-01           NaN  ...              NaN                 NaN   
        2016-03-02           NaN  ...              NaN                 NaN   
        2016-03-02           NaN  ...              NaN                 NaN   

                    likes_by_spillover_product_model  \
pd_code yymmdd                                         
309     2016-03-01                               NaN   
        2016-03-01                               NaN   
        2016-03-01                               NaN   
        2016-03-02                               NaN   
        2016-03-02                               NaN   

                    likes_by_spillover_product_only  total_likes_by_spillover  \
pd_code yymmdd                                                                  
309     2016-03-01                              NaN                       NaN   
        2016-03-01                              NaN                       NaN   
        2016-03-01                              NaN                       NaN   
        2016-03-02                              NaN                       NaN   
        2016-03-02                              NaN                       NaN   

                    avg_likes_product_model  avg_likes_product_only  \
pd_code yymmdd                                                        
309     2016-03-01                      NaN                     NaN   
        2016-03-01                      NaN                     NaN   
        2016-03-01                      NaN                     NaN   
        2016-03-02                      NaN                     NaN   
        2016-03-02                      NaN                     NaN   

                    avg_likes_spillover_product_model  \
pd_code yymmdd                                          
309     2016-03-01                                NaN   
        2016-03-01                                NaN   
        2016-03-01                                NaN   
        2016-03-02                                NaN   
        2016-03-02                                NaN   

                    avg_likes_spillover_product_only  \
pd_code yymmdd                                         
309     2016-03-01                               NaN   
        2016-03-01                               NaN   
        2016-03-01                               NaN   
        2016-03-02                               NaN   
        2016-03-02                               NaN   

                    avg_likes_total_spillover  
pd_code yymmdd                                 
309     2016-03-01                        NaN  
        2016-03-01                        NaN  
        2016-03-01                      

In [127]:
sls_reg.loc[222789].shape

(640, 61)

In [128]:
sls_reg["ori_pdcode"].value_counts()

22278    640
22152    640
2451     640
2449     640
22150    640
3690     640
300      640
9199     640
9189     640
2158     640
Name: ori_pdcode, dtype: int64

In [129]:
#sls_reg.to_excel("sales_data_regression.xlsx")

In [130]:
prod13 = list(sls_reg["ori_pdcode"].value_counts().index)

In [131]:
prod13

['22278',
 '22152',
 '2451',
 '2449',
 '22150',
 '3690',
 '300',
 '9199',
 '9189',
 '2158']

In [132]:
sls1 = sls_reg.reset_index().drop("pd_code",axis=1).sort_values(by=["ori_pdcode","yymmdd"])

In [133]:
#sls1["product_mention"]

In [134]:
#sls1.to_excel("sales_ads_plotting.xlsx", index=False)

In [135]:
sls1 = sls1.rename(columns={"product_model":"posting_pm", "product_only":"posting_po","model_only":"posting_mo",
                           'spillover_product_only':'posting_po_spillover','spillover_product_model':'posting_pm_spillover',
                           'total_spillover':'posting_spillover','product_mention':'numProduct_mention'})

In [136]:
strat = [x for x in sls1.columns if re.search("posting",x)]
#avgl = [x for x in sls1.columns if re.search("avg_likes",x) or re.search("max_likes",x)]

In [138]:
sls2 = sls1.set_index("ori_pdcode")
sls2["sales_yesterday"] = [np.nan] + list(sls2["sales_qt"][:-1])
#sls2["other_sales_yesterday"] = [np.nan] + list(sls2["other_sales_qt"][:-1])


In [139]:
prod13

['22278',
 '22152',
 '2451',
 '2449',
 '22150',
 '3690',
 '300',
 '9199',
 '9189',
 '2158']

In [140]:
drop_max = [x for x in sls2.columns if re.search("max_likes",x)]

In [141]:
afw=list()
for prod in prod13:
    fw = sls2.loc[prod]["yymmdd"].iloc[0]
    afw.append(fw)
    #fw = sls2.loc[prod]["Week"].iloc[1]
    #afw.append(fw)
fw = list(set(afw))

In [142]:
fw

[Timestamp('2016-03-01 00:00:00')]

In [143]:
sls3 = sls2.reset_index().set_index("yymmdd")
sls3 = sls3.drop(fw).drop(columns=drop_max)

In [144]:
sls3.dropna()["ori_pdcode"].value_counts()

22278    633
22152    633
2451     633
2449     633
22150    633
300      633
9199     633
9189     633
2158     633
Name: ori_pdcode, dtype: int64

In [145]:
#complete_sales_ads = sls3.drop(columns=["Year","Month","Date"]).dropna()
complete_sales_ads = sls3.drop(columns=["Year"]).dropna()

In [146]:
complete_sales_ads["ori_pdcode"].value_counts()

22278    633
22152    633
2451     633
2449     633
22150    633
300      633
9199     633
9189     633
2158     633
Name: ori_pdcode, dtype: int64

In [147]:
days_week = pd.read_excel("dummy_week_days.xlsx").set_index("yymmdd")
#days_week["dummy_Week"] = days_week["dummy_Week"]

In [148]:
df2 = complete_sales_ads.merge(days_week,how="left",on="yymmdd").drop_duplicates().reset_index().set_index(["ori_pdcode","yymmdd"])

In [149]:
df2.head()

sales_qt     buy_offer  buy_original  Week       Days  \
ori_pdcode yymmdd                                                              
2158       2016-03-08     154.0  1.891027e+06     3003000.0    10    Tuesday   
           2016-03-09     149.0  1.750582e+06     2905500.0    10  Wednesday   
           2016-03-10     175.0  1.955861e+06     3412500.0    10   Thursday   
           2016-03-11     163.0  1.929959e+06     3178500.0    10     Friday   
           2016-03-12     205.0  2.436869e+06     3997500.0    10   Saturday   

                           price_dc  price_reg  avg_numBrand  avg_numModel  \
ori_pdcode yymmdd                                                            
2158       2016-03-08  12279.393439    19500.0      0.021642      0.011298   
           2016-03-09  11748.869147    19500.0      0.020540      0.009885   
           2016-03-10  11176.350124    19500.0      0.022258      0.009885   
           2016-03-11  11840.239016    19500.0      0.021033      0.011298   
           2016-03-12  11887.164810    19500.0      0.018749      0.009885   

                       avg_numOYBrand  ...  likes_by_spillover_product_only  \
ori_pdcode yymmdd                      ...                                    
2158       2016-03-08             0.0  ...                              0.0   
           2016-03-09             0.0  ...                              0.0   
           2016-03-10             0.0  ...                              0.0   
           2016-03-11             0.0  ...                              0.0   
           2016-03-12             0.0  ...                              0.0   

                       total_likes_by_spillover  avg_likes_product_model  \
ori_pdcode yymmdd                                                          
2158       2016-03-08                       0.0                      0.0   
           2016-03-09                       0.0                      0.0   
           2016-03-10                       0.0                      0.0   
           2016-03-11                       0.0                      0.0   
           2016-03-12                       0.0                      0.0   

                       avg_likes_product_only  \
ori_pdcode yymmdd                               
2158       2016-03-08                     0.0   
           2016-03-09                     0.0   
           2016-03-10                     0.0   
           2016-03-11                     0.0   
           2016-03-12                     0.0   

                       avg_likes_spillover_product_model  \
ori_pdcode yymmdd                                          
2158       2016-03-08                                0.0   
           2016-03-09                                0.0   
           2016-03-10                                0.0   
           2016-03-11                                0.0   
           2016-03-12                                0.0   

                       avg_likes_spillover_product_only  \
ori_pdcode yymmdd                                         
2158       2016-03-08                               0.0   
           2016-03-09                               0.0   
           2016-03-10                               0.0   
           2016-03-11                               0.0   
           2016-03-12                               0.0   

                       avg_likes_total_spillover  sales_yesterday  dummy_Week  \
ori_pdcode yymmdd                                                               
2158       2016-03-08                        0.0            226.0          10   
           2016-03-09                        0.0            154.0          10   
           2016-03-10                        0.0            149.0          10   
           2016-03-11                        0.0            175.0          10   
           2016-03-12                        0.0            163.0          10   

                       dummy_Days  
ori_pdcode yymmdd                  
2158     

In [150]:
oy_display_brand = pd.read_excel("oy_display_brand.xlsx")
oy_sample = pd.read_excel("oy_sample.xlsx").set_index("pd_code")
oy_display_oth = pd.read_excel("oy_display_oth.xlsx").set_index("pd_code")

In [151]:
oy_display_oth.loc[2451].iloc[2]

start    20160901
end      20160930
Name: 2451, dtype: int64

In [152]:
for col in ["end","start"] :
    for df in [oy_sample,oy_display_oth]:
        df[col] = pd.to_datetime(df[col].astype(str)) 

In [153]:
prod13

['22278',
 '22152',
 '2451',
 '2449',
 '22150',
 '3690',
 '300',
 '9199',
 '9189',
 '2158']

In [154]:
oy_display_oth.index.unique()

Int64Index([2451, 2158, 2449, 9220], dtype='int64', name='pd_code')

In [155]:
oy_sample

,start,end
pd_code,,
22278,2016-02-01,2016-02-29
300,2017-06-01,2017-06-07
2242,2018-10-01,2018-10-31


In [156]:
df2["oy_sample"] =0
for i in oy_sample.index.unique():
    if str(i) not in prod13:
        #print("next")
        continue
    else:
        get_sample = oy_sample.loc[i]
        #print(get_sample)
        for row in range(len(get_sample)):
            period = get_sample.values
            date_range = pd.date_range(start=period[0],end=period[1])

            for dt in date_range:
                #print(str(i),dt)
                try:
                    df2.loc[(str(i),dt),"oy_sample"]=1

                except:
                    continue

In [157]:
df2.loc[df2["oy_sample"]!=0]

sales_qt     buy_offer  buy_original  Week       Days  \
ori_pdcode yymmdd                                                              
300        2017-06-01     352.0  7.418500e+06    13024000.0  74.0   Thursday   
           2017-06-02     350.0  7.368052e+06    12950000.0  74.0     Friday   
           2017-06-03     394.0  8.298246e+06    14578000.0  74.0   Saturday   
           2017-06-04     295.0  6.222831e+06    10915000.0  74.0     Sunday   
           2017-06-05     312.0  6.576038e+06    11544000.0  75.0     Monday   
           2017-06-06     254.0  5.368985e+06     9398000.0  75.0    Tuesday   
           2017-06-07     423.0  8.913869e+06    15651000.0  75.0  Wednesday   
22278      2016-02-01       NaN           NaN           NaN   NaN        NaN   
           2016-02-02       NaN           NaN           NaN   NaN        NaN   
           2016-02-03       NaN           NaN           NaN   NaN        NaN   
           2016-02-04       NaN           NaN           NaN   NaN        NaN   
           2016-02-05       NaN           NaN           NaN   NaN        NaN   
           2016-02-06       NaN           NaN           NaN   NaN        NaN   
           2016-02-07       NaN           NaN           NaN   NaN        NaN   
           2016-02-08       NaN           NaN           NaN   NaN        NaN   
           2016-02-09       NaN           NaN           NaN   NaN        NaN   
           2016-02-10       NaN           NaN           NaN   NaN        NaN   
           2016-02-11       NaN           NaN           NaN   NaN        NaN   
           2016-02-12       NaN           NaN           NaN   NaN        NaN   
           2016-02-13       NaN           NaN           NaN   NaN        NaN   
           2016-02-14       NaN           NaN           NaN   NaN        NaN   
           2016-02-15       NaN           NaN           NaN   NaN        NaN   
           2016-02-16       NaN           NaN           NaN   NaN        NaN   
           2016-02-17       NaN           NaN           NaN   NaN        NaN   
           2016-02-18       NaN           NaN           NaN   NaN        NaN   
           2016-02-19       NaN           NaN           NaN   NaN        NaN   
           2016-02-20       NaN           NaN           NaN   NaN        NaN   
           2016-02-21       NaN           NaN           NaN   NaN        NaN   
           2016-02-22       NaN           NaN           NaN   NaN        NaN   
           2016-02-23       NaN           NaN           NaN   NaN        NaN   
           2016-02-24       NaN           NaN           NaN   NaN        NaN   
           2016-02-25       NaN           NaN           NaN   NaN        NaN   
           2016-02-26       NaN           NaN           NaN   NaN        NaN   
           2016-02-27       NaN           NaN           NaN   NaN        NaN   
           2016-02-28       NaN           NaN           NaN   NaN        NaN   
           2016-02-29       NaN           NaN           NaN   NaN        NaN   

                           price_dc  price_reg  avg_numBrand  avg_numModel  \
ori_pdcode yymmdd                                                            
300        2017-06-01  21075.284091    37000.0      0.001181      0.010468   
           2017-06-02  21051.576923    37000.0      0.002097      0.012051   
           2017-06-03  21061.538462    37000.0      0.003767      0.012807   
           2017-06-04  21094.341591    37000.0      0.003767      0.011731   
           2017-06-05  21077.046351    37000.0      0.004656      0.014047   
           2017-06-06  21137.734706    37000.0      0.006176      0.017140   
           2017-06-07  21072.976905    37000.0      0.006264      0.020065   
22278      2016-02-01           NaN        NaN           NaN           NaN   
           2016-02-02           NaN        NaN           NaN           NaN   
           2016-02-03           NaN        NaN           NaN           NaN   
           2016-02-04           NaN        NaN

In [158]:
df2["oy_display_product"] =0
for i in oy_display_oth.index.unique():
    if str(i) not in prod13:
        #print("next")
        continue
    else:
        get_sample = oy_display_oth.loc[i]    
        for row in range(len(get_sample)):
            period = get_sample.iloc[row].values
            date_range = pd.date_range(start=period[0],end=period[1])
            for dt in date_range:
                #print(dt)
                #print(str(i),dt)
                df2.loc[(str(i),dt),"oy_display_product"]=1

          

In [159]:
df2.describe()

,sales_qt,buy_offer,buy_original,Week,price_dc,price_reg,avg_numBrand,avg_numModel,avg_numOYBrand,avg_numPos-Neg,...,total_likes_by_spillover,avg_likes_product_model,avg_likes_product_only,avg_likes_spillover_product_model,avg_likes_spillover_product_only,avg_likes_total_spillover,sales_yesterday,dummy_Week,oy_sample,oy_display_product
count,5697.000000,5.697000e+03,5.697000e+03,5697.000000,5697.000000,5697.000000,5697.000000,5697.000000,5697.000000,5697.000000,...,5697.000000,5697.000000,5697.000000,5697.000000,5697.000000,5697.000000,5697.000000,5697.000000,5726.000000,5726.000000
mean,169.666842,1.572639e+06,2.569680e+06,54.857820,12980.293829,20401.053186,0.013031,0.009038,0.000335,0.006088,...,1196.054064,200.588029,19.683848,57.832953,27.526027,114.567097,164.035808,54.857820,0.006287,0.058330
std,277.912808,2.142843e+06,3.625250e+06,26.108685,6059.486733,11207.964489,0.006591,0.009320,0.000994,0.005981,...,7017.995602,2868.015976,62.714732,1595.291889,199.860974,1568.486919,203.401028,26.108685,0.079049,0.234388
min,0.000000,0.000000e+00,0.000000e+00,10.000000,0.000000,0.000000,0.001181,0.000000,0.000000,-0.013531,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000
25%,49.000000,6.055192e+05,7.500000e+05,32.000000,8538.461538,13500.000000,0.008191,0.000000,0.000000,0.001751,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,49.000000,32.000000,0.000000,0.000000
50%,102.000000,1.316243e+06,2.080000e+06,55.000000,12901.822186,19500.000000,0.011816,0.007051,0.000000,0.004777,...,42.000000,0.000000,0.000000,0.000000,0.000000,0.000000,102.000000,55.000000,0.000000,0.000000
75%,184.000000,2.155838e+06,3.696000e+06,77.000000,16582.114927,32000.000000,0.016393,0.015023,0.000314,0.009029,...,252.000000,0.000000,0.000000,0.000000,0.000000,0.000000,183.000000,77.000000,0.000000,0.000000
max,11488.000000,8.559409e+07,1.503680e+08,100.000000,28105.769231,37000.000000,0.039805,0.040707,0.008392,0.031250,...,59071.000000,45465.000000,692.571429,45627.000000,4677.000000,45855.000000,2751.000000,100.000000,1.000000,1.000000


In [160]:
df2["oy_display_brand"] =0
for prod in prod13:
    for i in range(len(oy_display_brand)):
        period = oy_display_brand.iloc[i].values
        date_range = pd.date_range(start=period[0],end=period[1])
        for dt in date_range:
            df2.loc[(prod,dt),"oy_display_brand"]=1
   

In [161]:
df2.describe()

,sales_qt,buy_offer,buy_original,Week,price_dc,price_reg,avg_numBrand,avg_numModel,avg_numOYBrand,avg_numPos-Neg,...,avg_likes_product_model,avg_likes_product_only,avg_likes_spillover_product_model,avg_likes_spillover_product_only,avg_likes_total_spillover,sales_yesterday,dummy_Week,oy_sample,oy_display_product,oy_display_brand
count,5697.000000,5.697000e+03,5.697000e+03,5697.000000,5697.000000,5697.000000,5697.000000,5697.000000,5697.000000,5697.000000,...,5697.000000,5697.000000,5697.000000,5697.000000,5697.000000,5697.000000,5697.000000,5726.000000,5726.000000,5810.000000
mean,169.666842,1.572639e+06,2.569680e+06,54.857820,12980.293829,20401.053186,0.013031,0.009038,0.000335,0.006088,...,200.588029,19.683848,57.832953,27.526027,114.567097,164.035808,54.857820,0.006287,0.058330,0.051635
std,277.912808,2.142843e+06,3.625250e+06,26.108685,6059.486733,11207.964489,0.006591,0.009320,0.000994,0.005981,...,2868.015976,62.714732,1595.291889,199.860974,1568.486919,203.401028,26.108685,0.079049,0.234388,0.221308
min,0.000000,0.000000e+00,0.000000e+00,10.000000,0.000000,0.000000,0.001181,0.000000,0.000000,-0.013531,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000
25%,49.000000,6.055192e+05,7.500000e+05,32.000000,8538.461538,13500.000000,0.008191,0.000000,0.000000,0.001751,...,0.000000,0.000000,0.000000,0.000000,0.000000,49.000000,32.000000,0.000000,0.000000,0.000000
50%,102.000000,1.316243e+06,2.080000e+06,55.000000,12901.822186,19500.000000,0.011816,0.007051,0.000000,0.004777,...,0.000000,0.000000,0.000000,0.000000,0.000000,102.000000,55.000000,0.000000,0.000000,0.000000
75%,184.000000,2.155838e+06,3.696000e+06,77.000000,16582.114927,32000.000000,0.016393,0.015023,0.000314,0.009029,...,0.000000,0.000000,0.000000,0.000000,0.000000,183.000000,77.000000,0.000000,0.000000,0.000000
max,11488.000000,8.559409e+07,1.503680e+08,100.000000,28105.769231,37000.000000,0.039805,0.040707,0.008392,0.031250,...,45465.000000,692.571429,45627.000000,4677.000000,45855.000000,2751.000000,100.000000,1.000000,1.000000,1.000000


In [162]:
df3 = df2.dropna().reset_index()

In [163]:
df3[df3["oy_display_product"]!=0]

,ori_pdcode,yymmdd,sales_qt,buy_offer,buy_original,Week,Days,price_dc,price_reg,avg_numBrand,...,avg_likes_product_only,avg_likes_spillover_product_model,avg_likes_spillover_product_only,avg_likes_total_spillover,sales_yesterday,dummy_Week,dummy_Days,oy_sample,oy_display_product,oy_display_brand
54,2158,2016-05-01,264.0,2.987468e+06,5148000.0,17.0,Sunday,11316.165265,19500.0,0.010259,...,0.0,0.0,0.0,0.0,242.0,17.0,Sunday,0.0,1.0,0.0
55,2158,2016-05-02,193.0,2.098809e+06,3763500.0,18.0,Monday,10874.656324,19500.0,0.010259,...,0.0,0.0,0.0,0.0,264.0,18.0,Monday,0.0,1.0,0.0
56,2158,2016-05-03,184.0,2.001421e+06,3588000.0,18.0,Tuesday,10877.290148,19500.0,0.014935,...,0.0,0.0,0.0,0.0,193.0,18.0,Tuesday,0.0,1.0,0.0
57,2158,2016-05-04,276.0,3.028181e+06,5382000.0,18.0,Wednesday,10971.670018,19500.0,0.017924,...,0.0,0.0,0.0,0.0,184.0,18.0,Wednesday,0.0,1.0,0.0
58,2158,2016-05-05,193.0,2.002594e+06,3763500.0,18.0,Thursday,10376.132858,19500.0,0.018320,...,0.0,0.0,0.0,0.0,276.0,18.0,Thursday,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3610,2451,2017-05-27,233.0,3.548120e+06,8155000.0,73.0,Saturday,15227.981344,35000.0,0.005898,...,173.0,0.0,0.0,0.0,239.0,73.0,Saturday,0.0,1.0,0.0
3611,2451,2017-05-28,193.0,2.892316e+06,6755000.0,73.0,Sunday,14986.091228,35000.0,0.004687,...,173.0,0.0,0.0,0.0,233.0,73.0,Sunday,0.0,1.0,0.0
3612,2451,2017-05-29,240.0,3.739490e+06,8400000.0,74.0,Monday,15581.209056,35000.0,0.005022,...,173.0,0.0,0.0,0.0,193.0,74.0,Monday,0.0,1.0,0.0
3613,2451,2017-05-30,203.0,3.083069e+06,7105000.0,74.0,Tuesday,15187.530966,35000.0,0.001471,...,0.0,0.0,0.0,0.0,240.0,74.0,Tuesday,0.0,1.0,0.0


In [164]:
oy = [x for x in df3.columns if re.search ("oy",x)]
dummy_col = [x for x in df3.columns if re.search ("dummy",x)]
avg_likes = [x for x in df3.columns if re.search ("avg_likes",x) and not(re.search("week",x))]
max_likes = [x for x in df3.columns if re.search ("max_likes",x) and not(re.search("week",x))]
other_sales = [x for x in df3.columns if re.search ("other",x)]
num_ = [x for x in df3.columns if x.startswith("num")]

In [165]:
num_

['numProduct_mention']

In [166]:
other_sales

[]

In [167]:
strat

['posting_mo',
 'total_posting_this_day',
 'posting_pm',
 'posting_po',
 'posting_pm_spillover',
 'posting_po_spillover',
 'posting_spillover']

In [168]:
oy

['oy_sample', 'oy_display_product', 'oy_display_brand']

In [169]:
dummy_col

['dummy_Week', 'dummy_Days']

In [170]:

for col in dummy_col:
    df3[col] = df3[col].astype(str)

In [171]:
max_likes

[]

In [172]:
avg = [x for x in df3.columns if re.search ("avg",x) and not(re.search("likes",x))]

In [173]:
avg

['avg_numBrand',
 'avg_numModel',
 'avg_numOYBrand',
 'avg_numPos-Neg',
 'avg_Colorfulness_Picture',
 'avg_Saturation_Picture',
 'avg_Clarity_Picture',
 'avg_Brightness_Picture',
 'avg_Warm Hue_Picture',
 'avg_ROT_Picture',
 'avg_Diagonal_dominance_Picture',
 'avg_Physical_ver_Picture',
 'avg_Physical_Hor_Picture',
 'avg_Color_balance_vertical_Picture',
 'avg_Color_balance_horizontal_Picture',
 'avg_Texture_difference_Picture',
 'avg_Size_difference_Picture',
 'avg_Color_difference_Picture',
 'avg_Depth of field_hue_Picture',
 'avg_Depth of field_saturation_Picture',
 'avg_Depth of field_value_Picture',
 'avg_Contrast_Picture_z',
 'avg_Contrast_Picture^2',
 'avg_len_text_total',
 'avg_tag_numb',
 'avg_happiness',
 'avg_faces',
 'avg_image_size_kilobytes']

In [174]:
num_

['numProduct_mention']

In [175]:
avg_likes

['avg_likes_by_model_only',
 'avg_likes_product_model',
 'avg_likes_product_only',
 'avg_likes_spillover_product_model',
 'avg_likes_spillover_product_only',
 'avg_likes_total_spillover']

In [176]:
#"len_text_total"
#drop dc_ratio
df3_m1 = df3.loc[:,num_+other_sales+oy+max_likes+avg+avg_likes+dummy_col+strat+
                 ["yymmdd","sales_qt",
                  "sales_yesterday","ori_pdcode","price_dc"]]

In [177]:
#for data pm version 28/08
#df3_m1["avg_numProduct_mention"]

In [178]:
#'num_PdMention'
drop_this = ['posting_pm_spillover',
 'posting_po_spillover','avg_likes_spillover_product_model',
 'avg_likes_spillover_product_only','oy_display_brand',]

In [179]:
df4 = df3_m1.drop(columns=drop_this)

In [180]:
#df3_m1.describe()

In [181]:
#df3_m1.info(max_cols=1000)

In [182]:
#df3_m1.head()

In [183]:
df4["posting_po"].max()

37.0

In [184]:
df4["ori_pdcode"].value_counts()

22278    633
22152    633
2451     633
2449     633
22150    633
300      633
9199     633
9189     633
2158     633
Name: ori_pdcode, dtype: int64

In [185]:
df4.to_excel("oyof_sales_ads_daily_without_productspesificpromotion_20210111-tv.xlsx", index=False)

### 

In [186]:
df4.head()


,numProduct_mention,oy_sample,oy_display_product,avg_numBrand,avg_numModel,avg_numOYBrand,avg_numPos-Neg,avg_Colorfulness_Picture,avg_Saturation_Picture,avg_Clarity_Picture,...,posting_mo,total_posting_this_day,posting_pm,posting_po,posting_spillover,yymmdd,sales_qt,sales_yesterday,ori_pdcode,price_dc
0,0.0,0.0,0.0,0.021642,0.011298,0.0,0.006665,0.309184,0.229955,1.0,...,2.0,7.0,0.0,0.0,0.0,2016-03-08,154.0,226.0,2158,12279.393439
1,0.0,0.0,0.0,0.020540,0.009885,0.0,0.007434,0.310714,0.219127,1.0,...,2.0,8.0,0.0,0.0,0.0,2016-03-09,149.0,154.0,2158,11748.869147
2,0.0,0.0,0.0,0.022258,0.009885,0.0,0.009699,0.305357,0.220054,1.0,...,2.0,8.0,0.0,0.0,0.0,2016-03-10,175.0,149.0,2158,11176.350124
3,0.0,0.0,0.0,0.021033,0.011298,0.0,0.009826,0.321429,0.225367,1.0,...,2.0,7.0,0.0,0.0,0.0,2016-03-11,163.0,175.0,2158,11840.239016
4,0.0,0.0,0.0,0.018749,0.009885,0.0,0.011704,0.313393,0.210422,1.0,...,2.0,8.0,0.0,0.0,0.0,2016-03-12,205.0,163.0,2158,11887.164810


In [187]:
df4["avg_likes_product_model"].nlargest()

4340    45465.0
4341    45465.0
4342    45465.0
4343    45465.0
4344    45465.0
Name: avg_likes_product_model, dtype: float64